# Data Structures: Choosing the Right Tool

Choosing the right data structure is a critical design decision. Python offers a wide array of tools, from simple tuples to powerful libraries. This notebook explores the trade-offs between different options, helping you select the best tool for the job based on requirements for mutability, performance, and type safety.

## 1. Lightweight Data Structures

For many use cases, you don't need the overhead of a full class. Python's standard library provides several excellent options for creating simple, structured data containers.

In [1]:
from dataclasses import FrozenInstanceError, dataclass
from typing import NamedTuple, TypedDict


# Option 1: NamedTuple - Lightweight and immutable, but verbose
class PointNT(NamedTuple):
    x: int
    y: int


# Option 2: frozen dataclass - More features, still immutable
@dataclass(frozen=True)
class PointDC:
    x: int
    y: int


# Option 3: TypedDict - A dictionary with a fixed set of keys and value types
# It's just a type hint; at runtime, it's a regular dict.
class PointTD(TypedDict):
    x: int
    y: int


# --- Verification ---
p_nt = PointNT(1, 2)
p_dc = PointDC(1, 2)
p_td: PointTD = {"x": 1, "y": 2}

# Accessing data
assert p_nt.x == p_dc.x == p_td["x"] == 1

# Immutability check
raised_exception = None
try:
    p_nt.x = 3
except AttributeError as e:
    raised_exception = e
assert raised_exception is not None
print("✅ NamedTuple is immutable.")

raised_exception = None
try:
    p_dc.x = 3
except FrozenInstanceError as e:
    raised_exception = e
assert raised_exception is not None
print("✅ Frozen dataclass is immutable.")

# TypedDict is just a regular dict at runtime, so it IS mutable.
p_td["x"] = 3
assert p_td["x"] == 3
print("✅ TypedDict is mutable at runtime.")

✅ NamedTuple is immutable.
✅ Frozen dataclass is immutable.
✅ TypedDict is mutable at runtime.


| Feature | `NamedTuple` | `frozen dataclass` | `TypedDict` |
| :--- | :--- | :--- | :--- |
| **Mutability** | Immutable | Immutable | **Mutable** |
| **Runtime Object**| `tuple` subclass | `object` | `dict` |
| **Best For** | Simple, tuple-like value objects | Classes that shouldn't change after creation| Describing the shape of dictionaries (e.g., JSON data) |

## 2. Guaranteed & Performant Immutability with `pyrsistent`

**The Problem:** Even a `frozen dataclass` isn't truly immutable if it contains mutable structures like a `list`. A developer could accidentally mutate the list inside the dataclass, leading to hard-to-find bugs. Furthermore, comparing large, nested dictionaries for equality can be slow, and standard dictionaries cannot be used as keys in other dictionaries or sets because they are not hashable.

**The Solution:** We use `pyrsistent` for **guaranteed, efficient immutability**. It provides persistent data structures (`PMap` for dicts, `PVector` for lists). Any "update" to a `pyrsistent` object returns a new object, leaving the original untouched. 

### Key Use Case: Performant Equality Checks & Hashing

Thanks to a technique called **structural sharing**, `pyrsistent` is highly memory-efficient. When you "change" a value, it creates a new object but reuses all the unchanged parts of the old object. This also means that its hash is calculated very efficiently, based on the hashes of its contents.

This makes `pyrsistent` the ideal choice for:
1.  **Complex, long-lived state management** (e.g., the state of an application or an AI agent).
2.  **Using complex data as dictionary keys** (e.g., for caching/memoization).
3.  **Performant comparisons of large, nested data structures.** Comparing two `PMap`s is often much faster than deep-comparing two large standard `dict`s.

In [2]:
from pyrsistent import pmap

# --- Verification of Immutability ---
initial_state = pmap(
    {"user": "Alice", "settings": {"theme": "dark", "notifications": True}}
)

# `.set()` returns a NEW PMap instance
new_state = initial_state.set("user", "Bob")

assert initial_state is not new_state
assert initial_state["user"] == "Alice"
assert new_state["user"] == "Bob"
print("✅ `pmap.set` creates a new object, leaving the original untouched.")

# Structural sharing: the nested 'settings' pmap is shared between versions
assert initial_state["settings"] is new_state["settings"]
print("✅ Unchanged parts of the structure are shared in memory.")

# --- Verification of Hashing ---
state1 = pmap({"a": 1, "b": 2})
state2 = pmap({"a": 1, "b": 2})
state3 = pmap({"a": 1, "b": 99})

# PMaps with the same content have the same hash
assert state1 == state2
assert hash(state1) == hash(state2)
print("✅ Equal PMaps have equal hashes.")

# PMaps with different content have different hashes
assert state1 != state3
assert hash(state1) != hash(state3)
print("✅ Unequal PMaps have unequal hashes.")

# This allows them to be used as dictionary keys
cache = {state1: "Cached Value"}
assert cache[state2] == "Cached Value"  # Can look up with an equal PMap
print("✅ PMaps can be used as dictionary keys.")

# A standard dict cannot do this
raised_exception = None
try:
    d = {1: "a"}
    other_cache = {d: "Broken"}
except TypeError as e:
    raised_exception = e

assert raised_exception is not None
assert "unhashable type: 'dict'" in str(raised_exception)
print("✅ Standard dicts are correctly identified as unhashable.")

✅ `pmap.set` creates a new object, leaving the original untouched.
✅ Unchanged parts of the structure are shared in memory.
✅ Equal PMaps have equal hashes.
✅ Unequal PMaps have unequal hashes.
✅ PMaps can be used as dictionary keys.
✅ Standard dicts are correctly identified as unhashable.
